<a href="https://colab.research.google.com/github/tanumeena28/Ml-Ass1Q1/blob/main/lab2Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
uploaded = files.upload()

Saving faa_ai_prelim.csv to faa_ai_prelim.csv


In [4]:
df = pd.read_csv('faa_ai_prelim.csv',low_memory=False)

In [7]:
print("shape:",df.shape)
print("columns:",df.columns.tolist())

shape: (83, 42)
columns: ['UPDATED', 'ENTRY_DATE', 'EVENT_LCL_DATE', 'EVENT_LCL_TIME', 'LOC_CITY_NAME', 'LOC_STATE_NAME', 'LOC_CNTRY_NAME', 'RMK_TEXT', 'EVENT_TYPE_DESC', 'FSDO_DESC', 'REGIST_NBR', 'FLT_NBR', 'ACFT_OPRTR', 'ACFT_MAKE_NAME', 'ACFT_MODEL_NAME', 'ACFT_MISSING_FLAG', 'ACFT_DMG_DESC', 'FLT_ACTIVITY', 'FLT_PHASE', 'FAR_PART', 'MAX_INJ_LVL', 'FATAL_FLAG', 'FLT_CRW_INJ_NONE', 'FLT_CRW_INJ_MINOR', 'FLT_CRW_INJ_SERIOUS', 'FLT_CRW_INJ_FATAL', 'FLT_CRW_INJ_UNK', 'CBN_CRW_INJ_NONE', 'CBN_CRW_INJ_MINOR', 'CBN_CRW_INJ_SERIOUS', 'CBN_CRW_INJ_FATAL', 'CBN_CRW_INJ_UNK', 'PAX_INJ_NONE', 'PAX_INJ_MINOR', 'PAX_INJ_SERIOUS', 'PAX_INJ_FATAL', 'PAX_INJ_UNK', 'GRND_INJ_NONE', 'GRND_INJ_MINOR', 'GRND_INJ_SERIOUS', 'GRND_INJ_FATAL', 'GRND_INJ_UNK']


In [8]:
df["EVENT_DATETIME"] = pd.to_datetime(
    df["EVENT_LCL_DATE"].astype(str) + " " + df["EVENT_LCL_TIME"].astype(str),
    errors="coerce"
)

In [20]:
required_cols = [
    "ACFT_MAKE_NAME",
    "LOC_STATE_NAME",
    "ACFT_MODEL_NAME",
    "RMK_TEXT",
    "FLT_PHASE",
    "EVENT_TYPE_DESC",
    "FATAL_FLAG", # Add 'FATAL_FLAG' to the list of required columns
    "ACFT_DMG_DESC"
]

df_req = df[required_cols].copy()

In [21]:
txt = df_req["RMK_TEXT"].astype(str).str.upper()
fatal_keywords = ["FATAL", "KILL", "DIED", "DEATH", "FATALITY", "FATALLY"]
cond_fatal = txt.apply(lambda x: any(k in x for k in fatal_keywords))
df_req.loc[cond_fatal, "FATAL_FLAG"] = "Yes"
df_req["FATAL_FLAG"] = df_req["FATAL_FLAG"].fillna("No")


In [22]:
for col in ["FLT_PHASE", "ACFT_DMG_DESC"]:
    mode_val = df_req[col].mode(dropna=True)[0]
    df_req[col] = df_req[col].fillna(mode_val)


In [23]:
print("Missing values:\n", df_req.isnull().sum())

Missing values:
 ACFT_MAKE_NAME     5
LOC_STATE_NAME     0
ACFT_MODEL_NAME    4
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
dtype: int64


In [24]:
before_count = df_req.shape[0]
df_req = df_req[df_req["ACFT_MAKE_NAME"].notna() & (df_req["ACFT_MAKE_NAME"].str.strip() != "")]
df_req = df_req[df_req["ACFT_MODEL_NAME"].notna() & (df_req["ACFT_MODEL_NAME"].str.strip() != "")]
after_count = df_req.shape[0]
print(f"Dropped {before_count - after_count} rows with missing aircraft names.")


Dropped 6 rows with missing aircraft names.


In [25]:
threshold = 0.75 * len(df)
cols_to_drop = [col for col in df.columns if df[col].isnull().sum() > threshold]
df_req = df_req.drop(columns=[c for c in cols_to_drop if c in df_req.columns])


In [26]:
print(df_req.groupby("ACFT_MAKE_NAME").size().sort_values(ascending=False))


ACFT_MAKE_NAME
CESSNA                    23
PIPER                     10
BEECH                      9
MOONEY                     4
BOEING                     3
SOCATA                     2
LANCAIR                    2
CHAMPION                   2
BELL                       2
AIRBUS                     1
AEROSTAR INTERNATIONAL     1
AERONCA                    1
AERO COMMANDER             1
FAIRCHILD                  1
EMBRAER                    1
CONSOLIDATED VULTEE        1
CHRISTEN                   1
GRUMMAN                    1
FLIGHT DESIGN              1
GREAT LAKES                1
GLOBE                      1
MAULE                      1
HUGHES                     1
GULFSTREAM                 1
NORTH AMERICAN             1
PITTS                      1
SAAB                       1
SABRELINER                 1
VANS                       1
dtype: int64


In [43]:
print(df_req[df_req["FATAL_FLAG"] == "Yes"])


    ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0            BEECH  North Carolina              36   
53           PIPER         Florida            PA28   
55   FLIGHT DESIGN      California            CTLS   
79  NORTH AMERICAN         Arizona             F51   
80        CHAMPION      California           8KCAB   
81           BEECH      California              35   
82          CESSNA         Alabama             182   

                                             RMK_TEXT      FLT_PHASE  \
0   AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...  UNKNOWN (UNK)   
53  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...  UNKNOWN (UNK)   
55  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...  UNKNOWN (UNK)   
79  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...  UNKNOWN (UNK)   
80  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...  UNKNOWN (UNK)   
81  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...  UNKNOWN (UNK)   
82  N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...  UNKNOWN 

In [31]:
required_cols = [
    "ACFT_MAKE_NAME",
    "LOC_STATE_NAME",
    "ACFT_MODEL_NAME",
    "RMK_TEXT",
    "FLT_PHASE",
    "EVENT_TYPE_DESC",
    "FATAL_FLAG", # Ensure 'FATAL_FLAG' is included
    "ACFT_DMG_DESC"
]

df_req = df[required_cols].copy()

In [32]:
txt = df_req["RMK_TEXT"].astype(str).str.upper()
fatal_keywords = ["FATAL", "KILL", "DIED", "DEATH", "FATALITY", "FATALLY"]
cond_fatal = txt.apply(lambda x: any(k in x for k in fatal_keywords))
df_req.loc[cond_fatal, "FATAL_FLAG"] = "Yes"
df_req["FATAL_FLAG"] = df_req["FATAL_FLAG"].fillna("No")

In [33]:
for col in ["FLT_PHASE", "ACFT_DMG_DESC"]:
    mode_val = df_req[col].mode(dropna=True)[0]
    df_req[col] = df_req[col].fillna(mode_val)

In [34]:
print("Missing values:\n", df_req.isnull().sum())

Missing values:
 ACFT_MAKE_NAME     5
LOC_STATE_NAME     0
ACFT_MODEL_NAME    4
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
dtype: int64


In [35]:
before_count = df_req.shape[0]
df_req = df_req[df_req["ACFT_MAKE_NAME"].notna() & (df_req["ACFT_MAKE_NAME"].str.strip() != "")]
df_req = df_req[df_req["ACFT_MODEL_NAME"].notna() & (df_req["ACFT_MODEL_NAME"].str.strip() != "")]
after_count = df_req.shape[0]
print(f"Dropped {before_count - after_count} rows with missing aircraft names.")

Dropped 6 rows with missing aircraft names.


In [36]:
threshold = 0.75 * len(df)
# Keep 'FATAL_FLAG' column even if it has missing values as they were handled in a previous step
cols_to_drop = [col for col in df.columns if df[col].isnull().sum() > threshold and col != 'FATAL_FLAG']
df_req = df_req.drop(columns=[c for c in cols_to_drop if c in df_req.columns])

In [37]:
print(df_req[df_req["FATAL_FLAG"] == "Yes"])

    ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0            BEECH  North Carolina              36   
53           PIPER         Florida            PA28   
55   FLIGHT DESIGN      California            CTLS   
79  NORTH AMERICAN         Arizona             F51   
80        CHAMPION      California           8KCAB   
81           BEECH      California              35   
82          CESSNA         Alabama             182   

                                             RMK_TEXT      FLT_PHASE  \
0   AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...  UNKNOWN (UNK)   
53  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...  UNKNOWN (UNK)   
55  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...  UNKNOWN (UNK)   
79  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...  UNKNOWN (UNK)   
80  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...  UNKNOWN (UNK)   
81  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...  UNKNOWN (UNK)   
82  N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...  UNKNOWN 

In [38]:
dummies = pd.get_dummies(df_req["ACFT_DMG_DESC"], prefix="DMG")
dummies = dummies.iloc[:, :-1]  # drop last column
df_req = pd.concat([df_req.drop(columns=["ACFT_DMG_DESC"]), dummies], axis=1)


In [39]:
keywords = {
    "LANDING": "LANDING",
    "TAKEOFF": "TAKEOFF",
    "TAKE-OFF": "TAKEOFF",
    "TAKE OFF": "TAKEOFF",
    "CRUISE": "CRUISE",
    "APPROACH": "APPROACH",
    "CLIMB": "CLIMB",
    "DESCENT": "DESCENT",
    "TOUCHDOWN": "LANDING",
    "ROLL": "LANDING"
}

def extract_phase(text):
    text = str(text).upper()
    for k, v in keywords.items():
        if k in text:
            return v
    return "UNKNOWN"

df_req["FLIGHT_PHASE_TEXT"] = df_req["RMK_TEXT"].apply(extract_phase)


In [40]:
comparison = pd.crosstab(df_req["FLT_PHASE"], df_req["FLIGHT_PHASE_TEXT"])
print(comparison)


FLIGHT_PHASE_TEXT  APPROACH  LANDING  TAKEOFF  UNKNOWN
FLT_PHASE                                             
APPROACH (APR)            1        0        0        1
LANDING (LDG)             0       32        0       11
STANDING (STD)            0        0        0        3
TAKEOFF (TOF)             0        0        4        2
TAXI (TXI)                0        1        0        9
UNKNOWN (UNK)             0        0        0       13


In [42]:
print("Final shape:", df_req.shape)
df_req.to_csv("aviation_cleaned.csv", index=False)
print("Saved cleaned dataset to aviation_cleaned.csv")

Final shape: (77, 11)
Saved cleaned dataset to aviation_cleaned.csv
